In [3]:
import argparse
import logging
import os

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from train import train, train_and_evaluate
import matplotlib.pyplot as plt
from tifffile import imsave, TiffFile
from pathlib import Path
import PIL

import utils
import model.net as net
from evaluate import *

# import model.data_loader_tif as data_loader
import model.data_loader_png as data_loader

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Training with augmentation

When we have only 30 images the single most significant thing that we have to accomplish is to augment our data. Let's start with a few simple transformations and then we may dig deeper into custom transformations or packages. We use tranforms from [here](https://github.com/abarge/CS231n/blob/master/generator.py).

```python
# this is code from keras that we are going to reproduce
generator = image.ImageDataGenerator(rotation_range=0.2,
                fill_mode='nearest'
                width_shift_range=0.05,
                zoom_range=0.05,
                height_shift_range=0.05,
                shear_range=0.05,
                horizontal_flip=True)
```

### 1-rotation

We may do rotation with `RandomRotation` but we may also use `RandomAffine` - see below.

```python
transforms.RandomRotation(degrees=.2, resample=PIL.Image.NEAREST)
```

### 2-rotation-shift

```python
transforms.RandomAffine(degrees=.2, 
                        translate=(.05, .05), 
                        resample=PIL.Image.NEAREST)
```

### 3-rotation-shift-shear-flip

```python
transforms.RandomAffine(degrees=.2, 
                        translate=(.05, .05),
                        shear=.05,
                        resample=PIL.Image.NEAREST)
transforms.RandomHorizontalFlip()
```

### `params`

In [4]:
model_dir = 'experiments/augmented_model'
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)

In [5]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 3,
 'num_epochs': 25,
 'save_summary_steps': 100,
 'num_workers': 0}

### training

In [ ]:
!python3 train.py --model_dir 'experiments/augmented_model'

In [ ]:
!cat experiments/initial_model/train.log

### plots

Let's now plot accuracy and loss.

In [ ]:
history_path = os.path.join(model_dir, 'history.csv')

In [ ]:
df = pd.read_csv(history_path)

In [ ]:
df[['train_acc', 'val_acc']].plot();

In [ ]:
df[['train_loss', 'val_loss']].plot();

### prediction

In [ ]:
params.cuda = torch.cuda.is_available()
dataloaders = data_loader.fetch_dataloader(['train', 'test'], params)
train_loader = dataloaders['train']
test_dataloader = dataloaders['test']

In [ ]:
len(test_dataloader)

In [ ]:
model = net.Unet().cuda() if params.cuda else net.Unet()
checkpoint = os.path.join(model_dir, 'best.pth.tar')
utils.load_checkpoint(checkpoint, model);

In [ ]:
preds = predict_dataloader(model, test_dataloader, use_thresh=True)

In [ ]:
preds.shape

In [ ]:
preds[0, 0, :, :]

Let's save this `preds` as a `.tif` file.

In [ ]:
imsave(os.path.join(model_dir, 'preds.tif'), preds.reshape(30, 512, 512, 1))

Let's now visualize test image and prediction. 

In [ ]:
image, _ = test_dataloader.dataset[0]
plt.imshow(image.reshape(512, 512), cmap='gray');

In [ ]:
plt.imshow(preds[0].reshape(512, 512), cmap='gray');

### evaluation

The result is not very good - that' what we have to expect to get without augmentation. 

- `Rand Score Thin:` `0.872490253`
- `Information Score Thin: 0.960109659`